<a href="https://colab.research.google.com/github/bjkeulen/TM10007_PROJECT/blob/master/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM10007 Assignment template

In [8]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/bjkeulen/TM10007_PROJECT

In [0]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
 
# Metrics
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import r2_score

# Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [39]:
# Data loading functions. Uncomment the one you want to use
from hn.load_data import load_data

data = load_data()
features = data.drop('label', 1)
label = data['label']
label_int = label.str.replace('T','').astype(int)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

# Split labels in groups ??
# label[>X] = 0
# label[<=X] = 1

# Split in training and test data
feat_train, feat_test, label_train, label_test = train_test_split(
    features, label_int, test_size=0.4, random_state=0)

# Principal component analysis (PCA)
p = PCA(n_components=4)
p = p.fit(feat_train)
feat_train_pca = p.transform(feat_train)
feat_test_pca = p.transform(feat_test)

# Train and test classifiers, calculate performance
clsfs = [LinearDiscriminantAnalysis(),QuadraticDiscriminantAnalysis(),GaussianNB(), 
         LogisticRegression(),SGDClassifier(),KNeighborsClassifier()]
col_names = ['LDA', 'QDA', 'Gaussian NB', 'Logistic Regr', 'SGDC', 'KNN']
index_names = ['AUC', 'Accuracy'] # , 'F1 score''Precision', 'Recall']
performance = pd.DataFrame(index=[index_names])
print(performance)

N = 0
for clf in clsfs:
    # Train and test classifier
    clf.fit(feat_train_pca, label_train)
    pred = clf.predict(feat_test_pca)

    if hasattr(clf, 'predict_proba'):
    # The first column gives the probability for class = 0, so we take
    # the second which gives the probability class = 1:
        score = clf.predict_proba(feat_test_pca)[:, 1]
    else:
        score = pred

    # Performance
    auc = metrics.roc_auc_score(label_test, score)
    accuracy = metrics.accuracy_score(label_test, pred)
    # F1 = metrics.f1_score(label_test, pred)
    # precision = metrics.precision_score(label_test, pred)
    # recall = metrics.recall_score(label_test, pred)

    # Put into dataframe
    performance[col_names[N]] = [auc, accuracy] #, F1, precision, recall]
    N += 1

performance = performance.transpose()
print(performance)


The number of samples: 113
The number of columns: 160
Empty DataFrame
Columns: []
Index: [(AUC,), (Accuracy,)]
                    AUC  Accuracy
LDA            0.691871  0.586957
QDA            0.754253  0.739130
Gaussian NB    0.750473  0.695652
Logistic Regr  0.716446  0.500000
SGDC           0.543478  0.543478
KNN            0.770321  0.695652
